In [ ]:
import os
import cv2
import pygame
from ultralytics import YOLO
from time import sleep, time

# 현재 작업 디렉토리를 기준으로 경로 설정
BASE_DIR = os.getcwd()

# 모델 및 이미지 경로
MODEL_PATH = os.path.join(BASE_DIR, r"C:\Users\there\Desktop\SV\project2\archive\asl_yolov8_model.pt")
SIGN_IMAGE_DIR = os.path.join(BASE_DIR, r"C:\Users\there\Desktop\SV\project2\archive\양방향\양방향\sign_images")

# Pygame 초기화
pygame.init()

# 화면 크기 설정
screen = pygame.display.set_mode((800, 600))
pygame.display.set_caption("수화 텍스트 변환 및 수화 출력")

# 색상 정의
WHITE = (255, 255, 255)

# 알파벳 클래스 정의
classes = list("ABCDEFGHIJKLMNOPQRSTUVWXYZ")

# 수화 이미지 로드
sign_images = {}
for char in classes:
    image_path = os.path.join(SIGN_IMAGE_DIR, f"{char}.png")
    if os.path.exists(image_path):
        sign_images[char] = pygame.image.load(image_path)
    else:
        print(f"Warning: {image_path} not found!")

# YOLO 모델 로드
if os.path.exists(MODEL_PATH):
    model = YOLO(MODEL_PATH)
    print("YOLO 모델이 성공적으로 로드되었습니다.")
else:
    raise FileNotFoundError(f"YOLO 모델 파일을 찾을 수 없습니다: {MODEL_PATH}")

# 텍스트를 수화 이미지로 출력
def display_sign(text):
    """
    입력된 텍스트에 따라 수화 이미지를 화면에 표시.
    """
    screen.fill(WHITE)
    for char in text.upper():
        if char in sign_images:
            screen.fill(WHITE)
            image = pygame.transform.scale(sign_images[char], (400, 400))
            screen.blit(image, (200, 100))
            pygame.display.update()
            sleep(1)  # 1초 간격으로 표시
    pygame.display.update()

def main():
    # 카메라 열기
    cap = cv2.VideoCapture(0)
    if not cap.isOpened():
        print("카메라를 열 수 없습니다.")
        return

    print("Q 키를 눌러 종료하세요.")
    print("텍스트를 입력하려면 `T` 키를 누르세요.")
    detected_text = ""  # 수화를 통해 변환된 텍스트 저장
    max_text_length = 20  # 텍스트 길이 제한
    last_detected_char = None  # 마지막으로 인식된 문자
    last_detected_time = 0  # 마지막으로 문자를 감지한 시간
    delay = 1.0  # 입력 간 최소 딜레이(초)
    clock = pygame.time.Clock()  # FPS 제한용

    while True:
        ret, frame = cap.read()
        if not ret:
            print("프레임을 가져올 수 없습니다.")
            break

        # YOLOv8 예측
        results = model.predict(source=frame, conf=0.5, show=False, verbose=False)
        for result in results:
            for box in result.boxes:
                # 클래스 ID와 신뢰도
                class_id = int(box.cls[0])
                confidence = box.conf[0]

                # 신뢰도 임계값
                if confidence > 0.5:
                    detected_char = classes[class_id]

                    # 동일한 결과 반복 방지 및 딜레이 적용
                    current_time = time()
                    if detected_char != last_detected_char and (current_time - last_detected_time > delay):
                        last_detected_char = detected_char
                        last_detected_time = current_time
                        if len(detected_text) < max_text_length:
                            detected_text += detected_char

                    # 바운딩 박스 그리기
                    x1, y1, x2, y2 = map(int, box.xyxy[0])
                    cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
                    cv2.putText(frame, f"{detected_char} ({confidence:.2f})", (x1, y1 - 10),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)

        # 텍스트 표시
        cv2.putText(frame, f"Detected Text: {detected_text}", (10, 50),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)
        cv2.imshow("Sign to Text", frame)

        # Pygame 화면에서 수화 출력
        screen.fill(WHITE)
        if detected_text:
            char = detected_text[-1]  # 마지막 문자를 표시
            if char in sign_images:
                image = pygame.transform.scale(sign_images[char], (400, 400))
                screen.blit(image, (200, 100))
        pygame.display.update()

        # 키 입력 처리
        key = cv2.waitKey(1) & 0xFF
        if key == ord('q'):  # Q 키로 종료
            break
        elif key == ord('t'):  # T 키로 텍스트 입력 모드
            cap.release()
            cv2.destroyAllWindows()
            input_text = input("Enter a text to convert to sign language: ")
            display_sign(input_text)
            print("텍스트 수화 변환 완료. 프로그램으로 돌아갑니다.")
            cap = cv2.VideoCapture(0)  # 카메라 다시 열기

        clock.tick(30)  # FPS 제한

    cap.release()
    cv2.destroyAllWindows()
    pygame.quit()

if __name__ == "__main__":
    main()


pygame 2.6.1 (SDL 2.28.4, Python 3.12.8)
Hello from the pygame community. https://www.pygame.org/contribute.html
YOLO 모델이 성공적으로 로드되었습니다.
Q 키를 눌러 종료하세요.
텍스트를 입력하려면 `T` 키를 누르세요.
